<h2>ACTF</h2> 

<h>RSA LEAK</h>

<p>Task Description: We leak something for you~</p>

<p>Attachments:</p>


In [ ]:
from sage.all import *
from secret import flag
from Crypto.Util.number import bytes_to_long


def leak(a, b):
    p = random_prime(pow(2, 64))
    q = random_prime(pow(2, 64))
    n = p*q
    e = 65537
    print(f"new_n = {n}")
    print(f"leak = {(pow(a, e) + pow(b, e) + 0xdeadbeef) % n}")


def gen_key():
    while(True):
        a = randrange(0, pow(2,256))
        b = randrange(0, pow(2,256))
        p = pow(a, 4)
        q = pow(b, 4)
        rp = randrange(0, pow(2,24))
        rq = randrange(0, pow(2,24))
        pp = next_prime(p+rp)
        qq = next_prime(q+rq)
        if pp % pow(2, 4) == (pp-p) % pow(2, 4) and qq % pow(2, 4) == (qq-q) % pow(2, 4):
            print(f"rp, rq = {rp}, {rq}")
            print(f"pp, qq = {pp}, {qq}")
            n = pp*qq
            rp = pp-p
            rq = qq-q
            print(f"rp, rq = {rp}, {rq}")
            print(f"p, q = {p}, {q}")
            print("sdohla mat")
            return n, rp, rq

n, rp, rq = gen_key()
e = 65537
c = pow(bytes_to_long(flag), e, n)
print("n =", n)
print("e =", e)
print("c =", c)
leak(rp, rq)

'''
n = 3183573836769699313763043722513486503160533089470716348487649113450828830224151824106050562868640291712433283679799855890306945562430572137128269318944453041825476154913676849658599642113896525291798525533722805116041675462675732995881671359593602584751304602244415149859346875340361740775463623467503186824385780851920136368593725535779854726168687179051303851797111239451264183276544616736820298054063232641359775128753071340474714720534858295660426278356630743758247422916519687362426114443660989774519751234591819547129288719863041972824405872212208118093577184659446552017086531002340663509215501866212294702743
e = 65537
c = 48433948078708266558408900822131846839473472350405274958254566291017137879542806238459456400958349315245447486509633749276746053786868315163583443030289607980449076267295483248068122553237802668045588106193692102901936355277693449867608379899254200590252441986645643511838233803828204450622023993363140246583650322952060860867801081687288233255776380790653361695125971596448862744165007007840033270102756536056501059098523990991260352123691349393725158028931174218091973919457078350257978338294099849690514328273829474324145569140386584429042884336459789499705672633475010234403132893629856284982320249119974872840
=======leak=======
new_n = 122146249659110799196678177080657779971
leak = 90846368443479079691227824315092288065
'''

<p>First of all, let's talk about leak. As we can see the original rp and rq are very small, comparing tp p and q, and we expect the final rp and rq to have the same order(due to the density of the prime numbers). On the other hand, new n in the leak is not to big, and we are able to factor it out</p>

In [44]:
from multiprocessing import Pool 
from sage.all import factor
from gmpy2 import iroot, is_prime
from Crypto.Util.number import long_to_bytes

In [45]:
n1 = 122146249659110799196678177080657779971
leak = 90846368443479079691227824315092288065
e = 65537
print(factor(n1))

8949458376079230661 * 13648451618657980711


<p>Now we can brute force the rq and rp values:</p>

In [ ]:
p1, q1 = 8949458376079230661, 13648451618657980711
c1 = (leak - 0xdeadbeef) % n1
d1 = pow(e, -1, (p1 -1) * (q1 - 1))

num_threads = 20

def check(g):
    for x in g:
        a = pow(c1 - pow(x, e, n1), d1, n1)
        if(a <= 2**24 and ((pow(a, e, n1) + pow(x, e, n1)) % n1 == c1)):
            print(f"rp = {a}, rq = {x}")

gens = [range(i, 2**24+1, 32) for i in range(32)]
with Pool(num_threads) as pool:
    pool.map(check, gens)

rp = 11974933, rq = 405771


In [35]:
rp, rq = 11974933,  405771

<p>The second key idea is the relation between n and p*q. n = (p + rp) * (q + rq) = p * q + p * rq + q * rp + rp * rq. And we observe that p * q is a huge number, comparing to the rest part of n. p * q ~ 2^2048, when p * rq + q * rp + rp * rq ~ 2^(1024 + 24) * 2 = 2^1049</p>
<p>Also we know that p * q is a perfect 4th power of some number(a * b), let's compute the distance between two perfect 4th powers of these order:</p><p>(x + 1)^4 - x^4 = ((x + 1)^2 - x^2) * ((x+1)^2 + x^2) = x * (2*x + 1) * (2*x^2 + x + 1). x = a * b ~ 2^512 => (x + 1)^4 - x^4 ~ 2^512 * 2^513 * (2 * 2^1024 + 2^512 + 1) ~ 2^2050 > 2^1049 ~ p * rq + q * rp + rp * rq</p>Now we expect the round(n^(1/4)) to be the exact product of a and b!!!</p>

In [51]:
n = 3183573836769699313763043722513486503160533089470716348487649113450828830224151824106050562868640291712433283679799855890306945562430572137128269318944453041825476154913676849658599642113896525291798525533722805116041675462675732995881671359593602584751304602244415149859346875340361740775463623467503186824385780851920136368593725535779854726168687179051303851797111239451264183276544616736820298054063232641359775128753071340474714720534858295660426278356630743758247422916519687362426114443660989774519751234591819547129288719863041972824405872212208118093577184659446552017086531002340663509215501866212294702743
e = 65537
c = 48433948078708266558408900822131846839473472350405274958254566291017137879542806238459456400958349315245447486509633749276746053786868315163583443030289607980449076267295483248068122553237802668045588106193692102901936355277693449867608379899254200590252441986645643511838233803828204450622023993363140246583650322952060860867801081687288233255776380790653361695125971596448862744165007007840033270102756536056501059098523990991260352123691349393725158028931174218091973919457078350257978338294099849690514328273829474324145569140386584429042884336459789499705672633475010234403132893629856284982320249119974872840

In [52]:
ab = int(iroot(n, 4)[0])
pq = pow(ab, 4)

<p> Using the fact that N = (p + rp) * (q + rq), and the relating pq = p * q we can construct a quadratic equation:</p>
<p> N = pq + (pq / q) * rq + rp * q + rp * rq => q^2 * rp - (N - pq - rp * rq) * q + rq * pq = 0</p> 

In [53]:
a, b, c = rp, pq - n + rp * rq, rq * pq
D = b**2 - 4 * c * a
D = int(iroot(D, 2)[0])
if((-b + D) % (2 * a) == 0):
    q = (-b + D) // (2 * a)
else:
    q = (-b - D) // (2 * a)

In [60]:
p = pq // q
assert pq == p * q

In [67]:
pp = p + rp
qq = q + rq
assert is_prime(pp)
assert is_prime(qq)
assert pp * qq == n

phi = (pp - 1) * (qq - 1)
d = pow(e, -1, phi)
c = 48433948078708266558408900822131846839473472350405274958254566291017137879542806238459456400958349315245447486509633749276746053786868315163583443030289607980449076267295483248068122553237802668045588106193692102901936355277693449867608379899254200590252441986645643511838233803828204450622023993363140246583650322952060860867801081687288233255776380790653361695125971596448862744165007007840033270102756536056501059098523990991260352123691349393725158028931174218091973919457078350257978338294099849690514328273829474324145569140386584429042884336459789499705672633475010234403132893629856284982320249119974872840
m = pow(c, d, n)
print(long_to_bytes(m))

b'ACTF{lsb_attack_in_RSA|a32d7f}'
